In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from tqdm import tqdm

In [2]:
standing_url = "https://isu.html.infostradasports.com/cache/TheASP.asp@PageID%3D302901&SportID%3D302&TaalCode%3D2&StyleID%3D0&Cache%3D2.html?278331"

In [3]:
catalogue_data = requests.get(standing_url)

In [4]:
catalogue_soup = BeautifulSoup(catalogue_data.text).select("table")[0]

In [5]:
country_links_a = catalogue_soup.find_all("a")

In [6]:
# 尝试爬下来国家
# str(country_links_a[0])
# re.match(".+([A-Za-z]+)</span></a>", str(country_links_a[0])).group(1)

In [7]:
country_links = ["https://isu.html.infostradasports.com" + link_a.get("href") for link_a in country_links_a]

In [8]:
country_links = list(set(country_links))
len(country_links)

50

In [9]:
country_links

['https://isu.html.infostradasports.com/asp/redirect/isu.asp?page=athletebioslist_st&sportid=103&nocclubid=28&Cache=2',
 'https://isu.html.infostradasports.com/asp/redirect/isu.asp?page=athletebioslist_st&sportid=103&nocclubid=154&Cache=2',
 'https://isu.html.infostradasports.com/asp/redirect/isu.asp?page=athletebioslist_st&sportid=103&nocclubid=110&Cache=2',
 'https://isu.html.infostradasports.com/asp/redirect/isu.asp?page=athletebioslist_st&sportid=103&nocclubid=101&Cache=2',
 'https://isu.html.infostradasports.com/asp/redirect/isu.asp?page=athletebioslist_st&sportid=103&nocclubid=211&Cache=2',
 'https://isu.html.infostradasports.com/asp/redirect/isu.asp?page=athletebioslist_st&sportid=103&nocclubid=147&Cache=2',
 'https://isu.html.infostradasports.com/asp/redirect/isu.asp?page=athletebioslist_st&sportid=103&nocclubid=227&Cache=2',
 'https://isu.html.infostradasports.com/asp/redirect/isu.asp?page=athletebioslist_st&sportid=103&nocclubid=103&Cache=2',
 'https://isu.html.infostradaspor

In [10]:
athlete_table = []

In [11]:
for country_link in tqdm(country_links):
    athlete_data = requests.get(country_link)
    athlete_soup = BeautifulSoup(athlete_data.text).select("table")[0]
    athlete_links_a = athlete_soup.find_all("a")
    athlete_links = ["https://isu.html.infostradasports.com" + link_a.get("href") for link_a in athlete_links_a]
    for athlete_link in athlete_links:
        personal_data = requests.get(athlete_link)
        personal_soup = BeautifulSoup(personal_data.text)
        # 爬取姓名
        name = re.match("<h1>(.+)</h1>", str(personal_soup.select("#short > h1")[0])).group(1).title()
        
        # 爬取性别
        gender_data = personal_soup.select("#short > div > div.bio_personinfo_stats > table > tr:nth-child(5) > td.bio-text")[0]
        gender = re.match("<.+>([A-Za-z]+)</td>", str(gender_data)).group(1)
        # print(name, gender)
        
        # 爬取生日
        birth_data = personal_soup.select("#short > div > div.bio_personinfo_stats > table > tr:nth-child(4) > td.bio-text")[0]
        birth_year = re.match(".+(\d{4})", str(birth_data)).group(1)
        age = 2022 - int(birth_year)
        athlete_table.append([name, gender, birth_year, age])

100%|██████████| 50/50 [18:25<00:00, 22.11s/it]


In [12]:
name = ["Name", "Gender", "BirthYear", "Age"]
athletes = pd.DataFrame(columns=name, data=athlete_table)

In [13]:
# athletes.drop_duplicates().to_csv("data/new_athletes.csv", index=False)

In [15]:
athletes.to_csv("data/new_athletes.csv", index=False)